# Aim

In step four, I'd like to address expanding the phylogeny with more R2R3 MYB subfamiles to address some open questions: 
* Can I add a subfamily label to the other Azfi sequences I found before with a custom hmm.
* Can I infer the phylogeny to more confidently/explicitly draw the Salvinales VII branch as a part of the remaining VII branch by including other subfamiles and an outgroup

In a later notebook, I'll address adding extra data like: 
Next steps: 

* Can I add intron/exon structures as a second line of evidence supporting the different subfamilies
* Finally, I'd like to log2 expressen/fold change to the Azfi sequences; to illustrate how these subfamilies change expression upon transistion to sexual reproduction at least in _Azolla_

Since retrieving sequences from the different databases is quite a pain, 
I'll first retrieve only NCBI listed sequences (which I can retrieve in batch).
Subsequently, I may reitterate in this notebook, expanding on that analysis with sequences from other genome databases.

Genes in new list files in the `data` directory were retrieved from NCBI. 
Acc. nr.s from the J&R study were mixed protein and gene id's, so batch retrieval wasn't possible.
Fasta files with both acc nr's were created in the same directory.

# prepare data

Combining and linearising fasta files.

In [43]:
for i in data/*fasta
do  inseq=$(echo $i | cut -d '/' -f 2 | cut -d '.' -f 1)
    if   [ ! -f data/"$inseq"_linear.fasta ] 
    then cat data/$inseq.fasta \
         | awk '/^>/ {printf("%s%s\n",(N>0?"\n":""),$0);N++;next;} {printf("%s",$0);} END {printf("\n");}' \
         > data/"$inseq"_linear.fasta
    fi
done

In [3]:
ls data/*_linear.fasta

data/ARP_sequences_linear.fasta
data/Azfi-mybs-subfamVI-suspects_linear.fasta
data/Azfi-v1-MYB-sequences_linear.fasta
data/Azfi-v1-MYB-sequences_linear_linear.fasta
data/CDC5-outgroup_sequences_linear.fasta
data/combi_sequences_linear.fasta
data/combi_sequences_linear_linear.fasta
data/combi-VI-VII-Azfisuspects_linear.fasta
data/III_sequences_linear.fasta
data/II_sequences_linear.fasta
data/I_sequences_linear.fasta
data/IV_sequences_linear.fasta
data/MYB33_ARATH_linear.fasta
data/MYB33_ARATH_linear_linear.fasta
data/R1R2R3_sequences_linear.fasta
data/VIII_sequences_linear.fasta
data/VII_sequences_linear.fasta
data/VII_sequences_linear_linear.fasta
data/VI_sequences_linear.fasta
data/VI_sequences_linear_linear.fasta


In [45]:
cat data/CDC5-outgroup_sequences_linear.fasta \
    data/I_sequences_linear.fasta    \
    data/II_sequences_linear.fasta   \
    data/III_sequences_linear.fasta  \
    data/IV_sequences_linear.fasta   \
    data/V_sequences_linear.fasta    \
    data/VI_sequences_linear.fasta   \
    data/VII_sequences_linear.fasta  \
    data/VIII_sequences_linear.fasta \
    data/ARP_sequences_linear.fasta  \
    data/Azfi-mybs-subfamVI-suspects_linear.fasta \
    data/R1R2R3_sequences_linear.fasta \
    data/MYB33_ARATH_linear.fasta \
    > data/combi-I-to-VIII-Azfi_sequences_linear.fasta
    

In [5]:
inseq='combi-I-to-VIII-Azfi_sequences_linear'

In [50]:
conda activate phylogenetics
if    [ ! -d ./data/alignments_raw/ ]
then  mkdir  ./data/alignments_raw
fi
for   i in data/combi*sequences_linear.fasta
do    if    [ ! -f "./data/alignments_raw/$inseq"_aligned-mafft-linsi.fasta ]
      then  echo "aligning $inseq"
            linsi --thread $(nproc) data/$inseq.fasta > ./data/alignments_raw/"$inseq"_aligned-mafft-linsi.fasta \
                                                      2> ./data/alignments_raw/"$inseq"_aligned-mafft-linsi.log
      fi
done
conda deactivate

(phylogenetics) (phylogenetics) (phylogenetics) 

In [51]:
conda activate phylogenetics
einsi --thread $(nproc) data/$inseq.fasta > ./data/alignments_raw/"$inseq"_aligned-mafft-einsi.fasta \
                                                      2> ./data/alignments_raw/"$inseq"_aligned-mafft-einsi.log
conda deactivate

(phylogenetics) (phylogenetics) 

In [52]:
conda activate jalview
for   i in data/alignments_raw/*.fasta
do    prefix=$(echo $i | sed 's/\.fasta//')
      if    [ ! -f $prefix.png ]
      then  jalview -nodisplay \
                    -open $prefix.fasta \
                    -colour CLUSTAL \
                    -png  $prefix.png > /dev/null 2> /dev/null
      fi
done
conda deactivate

(jalview) (jalview) 

The linsi alignment looks like this:
![](./data/alignments_raw/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi.png)

And the einsi alignment looks like this:
![](./data/alignments_raw/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-einsi.png)

The linsi all. is even more gappy than the alignments we've seen before, but that stands to reason since a lot more of sequence diversity is covered in this MSA. 
After trimming this will look just fine I'm sure.

The einsi alignment seems to concentrate more sequence information into columns.
From this overview perspective, that looks quite good to me.

In [55]:
conda activate phylogenetics
if    [ ! -d data/alignments_trimmed ]
then  mkdir  data/alignments_trimmed 
fi

# define appendix only once here:
trimappendix='trim-gt4'

for a in "data/alignments_raw/$inseq"_aligned*.fasta
do  appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
    if    [ ! -f data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".fasta ]
    then  echo "trimming alignment $a"
          sed -i 's/ /_/g' $a
          trimal -in $a   \
                 -out data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".fasta \
                 -gt .4 \
                 -htmlout data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".html &
    fi
done
wait
conda deactivate

(phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) trimming alignment data/alignments_raw/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-einsi.fasta
[1] 172740
trimming alignment data/alignments_raw/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi.fasta
[2] 172747
(phylogenetics) [1]-  Done                    trimal -in $a -out data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".fasta -gt .4 -htmlout data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".html
[2]+  Done                    trimal -in $a -out data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".fasta -gt .4 -htmlout data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".html
(phylogenetics) 

In [56]:
conda activate jalview
for   i in data/alignments_trimmed/*.fasta
do    prefix=$(echo $i | sed 's/\.fasta//')
      if    [ ! -f $prefix.png ]
      then  jalview -nodisplay \
                    -open $prefix.fasta \
                    -colour CLUSTAL \
                    -png  $prefix.png > /dev/null 2> /dev/null &
      fi
done
wait
conda deactivate

(jalview) [1] 172804
[2] 172809
[3] 172823
[4] 172835
[5] 172842
[6] 172857
(jalview) [1]   Done                    jalview -nodisplay -open $prefix.fasta -colour CLUSTAL -png $prefix.png > /dev/null 2> /dev/null
[5]-  Done                    jalview -nodisplay -open $prefix.fasta -colour CLUSTAL -png $prefix.png > /dev/null 2> /dev/null
[2]   Done                    jalview -nodisplay -open $prefix.fasta -colour CLUSTAL -png $prefix.png > /dev/null 2> /dev/null
[3]   Done                    jalview -nodisplay -open $prefix.fasta -colour CLUSTAL -png $prefix.png > /dev/null 2> /dev/null
[4]-  Done                    jalview -nodisplay -open $prefix.fasta -colour CLUSTAL -png $prefix.png > /dev/null 2> /dev/null
[6]+  Done                    jalview -nodisplay -open $prefix.fasta -colour CLUSTAL -png $prefix.png > /dev/null 2> /dev/null
(jalview) 

### 40%
Starting at gap threshol 40%; the linsi alignment looks like this:
![](./data/alignments_trimmed/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi_trim-gt4.png)
and the einsi like so:
![](./data/alignments_trimmed/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-einsi_trim-gt4.png)

### 60%
Moving on to 60%; the linsi alignment looks like this:
![](./data/alignments_trimmed/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi_trim-gt6.png)
and the einsi like so:
![](./data/alignments_trimmed/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-einsi_trim-gt6.png)

### 80%
Finally at 80%; the linsi alignment looks like this:
![](./data/alignments_trimmed/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi_trim-gt8.png)
and the einsi like so:
![](./data/alignments_trimmed/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-einsi_trim-gt8.png)

Comparing these trimmings of both the einsi and the linsi allignment, I'm thinking that a raw or very leniently trimmed allignment retains the differentiating regions as described in J&R.
However, I also think that the more strictly trimmed regions may be better for phylogeny purposes.
Perhaps that the 80% is a bit too much, I'll loose some differentiating capability. 
So 60 is the middle ground then?
I'll run trees for all since I can miss the time and see what method gets me the best signal.
I'm putting my money on 60% einsi.

~~But first, I'll remove those two spurious seqeunces. from the alignments and trimms.~~
I have removed one spurious sequence and replaced the other with the correct one.
It appeared I had extracted a wrong sequence from ncbi.

# Tree inference

In [57]:
conda activate phylogenetics
for a in data/alignments_trimmed/"$inseq"_aligned*gt*.fasta
do  #iqpendix='iqtree-b100'
    iqpendix='iqtree-bb2000-alrt2000'



    file_appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
    
    if   [ ! -d    analyses/"$inseq"_trees/"$file_appendix" ]
    then echo "Making a directory $file_appendix to store trees (name based on alignment filename)"
         mkdir -p  analyses/"$inseq"_trees/"$file_appendix" 
    fi

    iqprefix=analyses/"$inseq"_trees/"$file_appendix"/"$inseq"_"$file_appendix"_"$iqpendix"
    if   [ ! -f "$iqprefix".treefile ]
    then echo "making a tree of file $a"
         nice iqtree -s $a \
                     -m MFP \
                     -bb 2000 -alrt 2000 \
                     -nt AUTO \
                     -ntmax $(nproc)  \
                     -pre  "$iqprefix" \
                     2>   "$iqprefix".stderr \
                      >    "$iqprefix".stdout
    #cat "$iqprefix".log | mail -s "IQtree_run $a" laura
    fi
done
conda deactivate

(phylogenetics) Making a directory aligned-mafft-einsi_trim-gt4 to store trees (name based on alignment filename)
making a tree of file data/alignments_trimmed/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-einsi_trim-gt4.fasta
Making a directory aligned-mafft-einsi_trim-gt6 to store trees (name based on alignment filename)
making a tree of file data/alignments_trimmed/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-einsi_trim-gt6.fasta
Making a directory aligned-mafft-einsi_trim-gt8 to store trees (name based on alignment filename)
making a tree of file data/alignments_trimmed/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-einsi_trim-gt8.fasta
making a tree of file data/alignments_trimmed/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi_trim-gt4.fasta
Making a directory aligned-mafft-linsi_trim-gt6 to store trees (name based on alignment filename)
making a tree of file data/alignments_trimmed/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi_trim-gt6.fas

This [initial tree](https://itol.embl.de/tree/1312115964244391596537363#) based on the `combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi_trim-gt4` alignment, puts some sequences at odd places. 
I'm wondering if I have a too gappy alignment here, like I already suspected for earlier alignments.
Alternativelly, I may try to align sequences with the einsi method in stead of the linsi method, since the big conserved domain is quite gappy and linsi may not align conserved sites outside of the main domain propperly.

On second thought, I reaslise that some sequences are very oddly aligned.
Perhaps I made a typo extracting these, or they could be incomplete.

# conclusions

Comparing the different trimming percentages, it seems that 60% minumum sequence content per column gives me the better internal consistency and phylogenetic signal than the 40% alignments for both einsi and linsi.

In the 80% tree, some branch lengths understandibly become versy short. 
Bootstraps are similar to the 60% tree.

The ARP clade isn't consistently placed and since it is not of interest for me, I would opt to take it out.
Similarly, one CDC5 sequence behaves odly: it is consistently placed with subfamily VIII sequences.
I suspect it was miscategorised and I will omit it from this analysis.

Comparing the einsi to the linsi alignment, I find that the linsi alignment places the ARP clade often on very early branches, while the einsi alignment puts it often with the VII category of sequences. 
Topology wise, thei einsi method reproduced the structure of older clades as shown in J&R, but the linsi method has better internal consistency in those regions. 
Alternativelly, the ARP clade may weaken internal consitency in whatever regions it pops up.
In that case I'm overinterpreting a side effect of including this oddly behaving clade of sequences.
I'll reinspect the MSAs and choose a method. 
Additionally, I'll run non-parametric trees on both data sets but remove the ARP sequences before.

Biologically speaking, the Azfi sequences of interest end up in the same clades as they did before. 
The VII salviniales clade is placed nicely with the other VII sequences with OK support.
The newly introduced R2R3 myb sequences from the Azfi genome (result from the hmm scan) are all placed in the VIII subfamily; answering that question. 
This meets the two aims I set for this notebook; but I'll double check still with two nonparametric tree runs.


In [3]:
cd ./analyses/combi-I-to-VIII-Azfi_sequences_linear_trees/
grep 'Best-fit model' */*.log
cd ../../

aligned-mafft-einsi_trim-gt4/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-einsi_trim-gt4_iqtree-bb2000-alrt2000.log:Best-fit model: VT+F+R5 chosen according to BIC
aligned-mafft-einsi_trim-gt6/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-einsi_trim-gt6_iqtree-bb2000-alrt2000.log:Best-fit model: VT+R5 chosen according to BIC
aligned-mafft-einsi_trim-gt8/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-einsi_trim-gt8_iqtree-bb2000-alrt2000.log:Best-fit model: LG+I+G4 chosen according to BIC
aligned-mafft-linsi_trim-gt4/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi_trim-gt4_iqtree-bb2000-alrt2000.log:Best-fit model: LG+F+R8 chosen according to BIC
aligned-mafft-linsi_trim-gt6/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi_trim-gt6_iqtree-bb2000-alrt2000.log:Best-fit model: LG+F+I+G4 chosen according to BIC
aligned-mafft-linsi_trim-gt8/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi_trim-gt8_iqtree-bb2000-alrt2000.log:Best-fit model: LG+G4 

In [6]:
conda activate phylogenetics
for a in data/alignments_trimmed/"$inseq"_aligned*gt6-seqrmmanual.fasta
do  iqpendix='iqtree-b200'
    #iqpendix='iqtree-bb2000-alrt2000'

    file_appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
    
    if   [ ! -d    analyses/"$inseq"_trees/"$file_appendix" ]
    then echo "Making a directory $file_appendix to store trees (name based on alignment filename)"
         mkdir -p  analyses/"$inseq"_trees/"$file_appendix" 
    fi

    iqprefix=analyses/"$inseq"_trees/"$file_appendix"/"$inseq"_"$file_appendix"_"$iqpendix"
    if   [ ! -f "$iqprefix".treefile ]
    then echo "making a tree of file $a"
         nice iqtree -s $a  \
                     -m MFP \
                     -b 200 \
                     -nt AUTO \
                     -ntmax $(nproc)  \
                     -pre  "$iqprefix" \
                     2>   "$iqprefix".stderr \
                      >    "$iqprefix".stdout
    #cat "$iqprefix".log | mail -s "IQtree_run $a" laura
    fi
done
conda deactivate

(phylogenetics) Making a directory aligned-mafft-einsi_trim-gt6-seqrmmanual to store trees (name based on alignment filename)
making a tree of file data/alignments_trimmed/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-einsi_trim-gt6-seqrmmanual.fasta
Making a directory aligned-mafft-linsi_trim-gt6-seqrmmanual to store trees (name based on alignment filename)
making a tree of file data/alignments_trimmed/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi_trim-gt6-seqrmmanual.fasta
(phylogenetics) 

# conclusions
Inspecting the two nonparametric trees, they are actually very similar and allow for some solid biological interpretation now.

I am convinced that ferns do have the VI subfamily R2R3 Myb, in contradiction to J&R. 
The Azolla sequences of interest are confidently placed in the VII subfamily.
It could be fun to see if there is more Azfi R2R3 mybs I could put in this tree; it's not often you can search a fern genome for such orthologous and examine their differential expressen in context of the fern life cycle.

### technical stuff.
The spurious CDC5 sequence should go out, it is all over the place and I doubt it is a CDC5 at all.

I added one arabidopsis sequence, it may be nice to add a couple well known ones to the different clades to illustrate their possible function.

Although the linsi alignment gives me trees with better supported nodes on average (regardless of their position in the tree) the einsi alignment gives me better support for the ancestral nodes.
Looking at the alignments again (trimmed at 60% collumn content) I think to see that the einsi alignments more consistently aligns the part just after the main conserved domain.


Let's add some snapshots of the nonparametric trees in here, first the einsi based tree: [(link)](https://itol.embl.de/tree/1312115964248471596621955)
![einsi](analyses/combi-I-to-VIII-Azfi_sequences_linear_trees/aligned-mafft-einsi_trim-gt6-seqrmmanual/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-einsi_trim-gt6-seqrmmanual_iqtree-b200.png)

And then the linsi tree [link](https://itol.embl.de/tree/1312115964249831596621961)
![einsi](analyses/combi-I-to-VIII-Azfi_sequences_linear_trees/aligned-mafft-linsi_trim-gt6-seqrmmanual/combi-I-to-VIII-Azfi_sequences_linear_aligned-mafft-linsi_trim-gt6-seqrmmanual_iqtree-b200.png)